In [1]:
#default_exp interference

In [2]:
import sys
sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")

In [3]:
#export
import torch
import logging

from pointpillars.utils.io import read_config, load_network_save, load_single_pcd
from pointpillars.data.dataset import VelTrainDataset, collate_fn
from pointpillars.modules.pointpillars import PointPillars, init_weights
from pointpillars.utils.pcviewer import PCViewer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
#export
logger = logging.getLogger(__name__)


In [5]:
#export
def predict_single_pcd(pcd_nbr: int, net_save_name: str):
    """Create a network prediction from a saved network for a single point cloud"""

    vel_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
    label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"

    dataset = VelTrainDataset(vel_folder, label_folder)
    batch_tuple = dataset[pcd_nbr]
    
    batch = []
    for i, tensor in enumerate(batch_tuple):
        tensor = tensor.unsqueeze(0)
        batch.append(tensor)
        
    pil_batch, ind_batch, label_batch = batch

    #pil_batch, ind_batch, label_batch, _ = collate_fn(batch)
    with torch.no_grad():
        conf = read_config()
        net_save = load_network_save(net_save_name)
        net = PointPillars(conf)
        net.load_state_dict(net_save)
        net.eval()
        net.cuda()

        pred_anchors, label_mask = net(pil_batch, ind_batch, label_batch)

    return pred_anchors, label_batch, label_mask

In [6]:
#export
def bb_tensor_to_velodyne_coords(bb_cam: torch.Tensor):
    xc, yc, zc, box_dims = bb_cam[:,0], bb_cam[:, 1], bb_cam[:, 2], bb_cam[:, 3:]

    xv = zc
    yv = -1 * xc
    zv = -1 * yc

    return torch.cat((xv.unsqueeze(1), yv.unsqueeze(1), zv.unsqueeze(1), box_dims), dim=1)


In [7]:
#export
def view_single_pcd_prediction(pcd_nbr: str, net_save_name: str):
    pcloud = load_single_pcd(pcd_nbr)
    pred_anchors, label_batch, label_mask = predict_single_pcd(int(pcd_nbr), net_save_name)
    
    #xp, yp, zp, rp = pcloud[:,0], pcloud[:,1], pcloud[:,2], pcloud[:,3]
    #pcloud = torch.stack((xp, yp, zp, rp), dim=1)
    #pcloud = torch.stack((zp, yp, xp, rp), dim=1)
    label_batch = label_batch[0]
    #pred_anchors = bb_tensor_to_velodyne_coords(pred_anchors)
    label_batch = bb_tensor_to_velodyne_coords(label_batch)

    PCViewer.view_pcd_from_network_output(pcloud, pred_anchors, label_batch)





In [8]:
pcloud = load_single_pcd('002137')
pcloud

tensor([[ 7.7512e+01,  4.7000e-02,  2.8300e+00,  0.0000e+00],
        [ 5.2519e+01,  8.8510e+00,  2.0100e+00,  1.6000e-01],
        [ 5.2292e+01,  8.9810e+00,  2.0030e+00,  1.0000e-01],
        ...,
        [ 3.8420e+00, -1.4470e+00, -1.7980e+00,  3.9000e-01],
        [ 3.8530e+00, -1.4380e+00, -1.8020e+00,  0.0000e+00],
        [ 3.8750e+00, -1.4320e+00, -1.8100e+00,  0.0000e+00]], device='cuda:0')

In [9]:
torch.min(pcloud, dim=0), torch.max(pcloud, dim=0)

(torch.return_types.min(
 values=tensor([-71.8370, -55.1200,  -8.8140,   0.0000], device='cuda:0'),
 indices=tensor([ 13281,   1474, 113923,      0], device='cuda:0')),
 torch.return_types.max(
 values=tensor([79.8020, 32.9710,  2.8300,  0.9900], device='cuda:0'),
 indices=tensor([ 6116, 10917,     0, 28486], device='cuda:0')))

In [ ]:
view_single_pcd_prediction("000141", "trained_22-06-2021-19-43-55_e10000")


INFO - 2021-06-23 16:23:00,235 - dataset - Initializing velodyne training dataset...
INFO - 2021-06-23 16:23:00,243 - compute_pillars - Calculating Pillars...
INFO - 2021-06-23 16:23:00,244 - compute_pillars - Selecting points in pillar...
INFO - 2021-06-23 16:23:00,253 - compute_pillars - Removing invalid points from point cloud...
INFO - 2021-06-23 16:23:00,710 - compute_pillars - Pillar calculation complete!
INFO - 2021-06-23 16:23:00,711 - compute_pillars - Calculating Pillars..
INFO - 2021-06-23 16:23:00,715 - compute_pillars - Calculating pillar_centers_from_index.
INFO - 2021-06-23 16:23:00,795 - head - Initialize detection head...
INFO - 2021-06-23 16:23:00,798 - boxmatch - Initializing BoxMatch module...
INFO - 2021-06-23 16:23:01,063 - head - Forward through detection head...
INFO - 2021-06-23 16:23:01,117 - boxmatch - Forward pass through BoxMatch...
INFO - 2021-06-23 16:23:01,147 - boxmatch - Calculating absolute_boxes_from_anchors...
